# Form Parsing

<img src="images/form_parsing_with_human_feeadback.png" width="500"/>

In [1]:
import os, json
from llama_parse import LlamaParse
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import (
    VectorStoreIndex
    , StorageContext
    , load_index_from_storage
)
from llama_index.core.workflow import (
    StartEvent
    , StopEvent
    , Workflow
    , step
    , Event
    , Context
    , InputRequiredEvent
    , HumanResponseEvent
)
from helper import get_llama_cloud_api_key, get_llama_cloud_base_url, extract_html_content
from IPython.display import display, HTML
from llama_index.utils.workflow import draw_all_possible_flows


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
llama_cloud_api_key = get_llama_cloud_api_key()
llama_cloud_base_url = get_llama_cloud_base_url()

In [4]:
class ParseFormEvent(Event):
    application_form: str

class QueryEvent(Event):
    query: str
    field: str 

class ResponseEvent(Event):
    response: str

class FeedbackEvent(Event):
    feedback: str

class GenerateQuestionEvent(Event):
    pass

In [5]:
class RAGWorkflow(Workflow):
    storage_dir = "./storage"
    llm: Ollama
    query_engine: VectorStoreIndex
    embed_model: OllamaEmbedding

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:
        if not ev.resume_file:
            raise ValueError("No resume file provided")
        
        # define LLM
        self.llm = Ollama(model="llama3.2:3b", request_timeout=5000.0)
        self.embed_model=OllamaEmbedding(model_name="nomic-embed-text", request_timeout=5000.0)

        if os.path.exists(self.storage_dir):
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context, embed_model=self.embed_model)
        else:
            documents = LlamaParse(
                result_type="markdown"
                , api_key=llama_cloud_api_key
                , base_url=llama_cloud_base_url
                , content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)

            index = VectorStoreIndex.from_documents(
                documents=documents
                , embed_model=self.embed_model
            )
            index.storage_context.persist(persist_dir=self.storage_dir)
        
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        return ParseFormEvent(application_form=ev.application_form)
    
    @step
    async def parse_form(self, ctx: Context, ev:ParseFormEvent) -> GenerateQuestionEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key
            , base_url=llama_cloud_base_url
            , result_type="markdown"
            , content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in."
            , formatting_instruction="Return a bulleted list of the fields ONLY."
        )

        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            f"""
            This is a parsed form.
            Convert it into a JSON object containing only the list 
            of fields to be filled in, in the form {{ fields: [...] }}. 
            <form>{result.text}</form>.
            Return JSON ONLY, no markdown."""
        )
        print(raw_json)
        fields = json.loads(raw_json.text)["fields"]
        
        await ctx.set("fields_to_fill", fields)
        return GenerateQuestionEvent()
    
    @step
    async def generate_question(self, ctx: Context, ev: GenerateQuestionEvent | FeedbackEvent) -> QueryEvent:
        fields = await ctx.get("fields_to_fill")

        for field in fields:
            question = f"How would you answer this question about the candidate? <field>{field}</field>"

            if hasattr(ev, "feedback"):
                question += f"""
                    \nWe previously got feedback about how we answered the questions.
                    It might not be relevant to this particular field, but here it is:
                    <feedback>{ev.feedback}</feedback>
                """
            
            ctx.send_event(QueryEvent(field=field, query=question))

            await ctx.set("total_fields", len(fields))

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return ResponseEvent(field=ev.field, response=response.response)
    
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> InputRequiredEvent:
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None 
        
        responseListString = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(
            f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseListString}
            </responses>
            """
        )
        
        await ctx.set("filled_form", str(result))

        return InputRequiredEvent(
            prefix="\nHow does this look? Give me any feedback you have on any of the answers."
            , result=result
        )

    @step
    async def get_feedback(self, ctx: Context, ev: HumanResponseEvent) -> FeedbackEvent | StopEvent:
        result = self.llm.complete(
            f"""
            You have received some human feedback on the form-filling task you've done.
            Does everything look good, or is there more work to be done?
            <feedback>
            {ev.response}
            </feedback>
            If everything is fine, respond with just the word 'OKAY'.
            If there's any other feedback, respond with just the word 'FEEDBACK'.
            """
        )
        print(f">>>> Raw result verdict: {result}")

        verdict = result.text.strip()
        print(f"LLM says the verdict was {verdict}")
        if verdict == "OKAY":
            return StopEvent(result=await ctx.get("filled_form"))
        else:
            return FeedbackEvent(feedback=ev.response)

In [ ]:
w = RAGWorkflow(verbose=False, timeout=5000)
handler = await w.run(
    resume_file="data/fake_resume.pdf"
    , application_form="data/fake_application_form.pdf"
)

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        print("We've filled in your form! Here are the results:\n")
        print(event.result)

        response = input(event.prefix)
        handler.ctx.send_event(HumanResponseEvent(response=response))

response = await handler
print("Agent complete! Here's your final result:")
print(str(response))

Started parsing the file under job_id 7b3be277-c79f-4637-8a50-856c6ca5938a
{"fields":["First Name","Last Name","Email","Phone","Linkedin","Project Portfolio","Degree","Graduation Date","Current Job title","Technical Skills","Describe why you’re a good fit for this position","Do you have 5 years of experience in React"]}


In [ ]:
WORKFLOW_FILE = "workflows/form_parsing_workflow.html"
draw_all_possible_flows(w, filename=WORKFLOW_FILE)
html_content = extract_html_content(WORKFLOW_FILE)
display(HTML(html_content), metadata=dict(isolated=True))